In [1]:
import pandas as pd
import geopandas
import numpy as np
from geopandas.tools import sjoin
import folium
import googlemaps
from folium.plugins import MarkerCluster
from branca.colormap import linear

In [2]:
def get_current_call(file):
    calls = pd.read_csv(file,parse_dates=[3]) 
    current_call = calls.loc[calls.TIME == max(calls.TIME)]
    return current_call.values[0]

def get_api_key(filepath):
    with open(filepath) as f:
        content = f.readlines()
    # remove whitespace characters at the end of each line
    content = [x.strip() for x in content]
    key = content[0]
    return key

def api_call(amb_coord,call_coord,dep_time,key,available_amb):
    gmaps = googlemaps.Client(key=key)
    result = gmaps.distance_matrix(amb_coord, call_coord, mode="driving", units="imperial", departure_time=dep_time)
    output_mat = pd.DataFrame()
    for idx, row in enumerate(result['rows']):
        row_mat = pd.DataFrame()
        mat = row['elements'][0]
        for key, val in mat.items():
            if key != 'status':
                df = pd.DataFrame.from_dict(val, orient='index')
                df = df.transpose()
                df.columns = [key + "_" + c for c in df.columns]
                if row_mat.empty:
                    row_mat = df
                else:
                    row_mat = pd.concat([row_mat, df], axis=1)
        if output_mat.empty:
            output_mat = row_mat
        else:
            output_mat = output_mat.append(row_mat)
    output_mat.index = [amb for amb in available_amb.AMB_ID]
    
    chosen = output_mat.loc[output_mat.duration_in_traffic_value == min(output_mat.duration_in_traffic_value)].index[0]
    return chosen

In [3]:
def add_ambmarker(smap,lat,long,amb_name):
    folium.Marker([lat, long], icon=folium.Icon(icon='plus',color='blue'),
                  popup="Amb #:" + amb_name
                 ).add_to(smap)
    
def dispatch_ambmarker(smap,lat,long,amb_name):
    folium.Marker([lat, long], icon=folium.Icon(icon='plus',color='gray'),
                  popup="Amb #:" + amb_name
                 ).add_to(smap)
    
def add_emsmarker(smap,lat,long,event_id):
    folium.RegularPolygonMarker([lat, long], popup="EMS #:" + event_id,
                                fill_color='red',number_of_sides=5,radius=10).add_to(smap)


In [4]:
def create_map(gfile,ambfile,current_call,api_key):  
    #geopandas
    gdf = geopandas.read_file(gfile)
    gdf["prob"] = np.random.random_sample(len(gdf))
    gdf1 = gdf
    gdf = gdf.set_index('ZCTA5CE10')['prob']
    gdf1 = gdf1.set_index('ZCTA5CE10')
    gdf1.crs={'init': 'epsg:4326'}
    
    #color scale
    colormap = linear.OrRd.scale(0.0,1.0)
    
    #foliium
    sfmap = folium.Map([37.7556, -122.4399], zoom_start = 12)
    
    #plot zip codes and prob color grid
    folium.GeoJson(gdf1.to_json(),overlay=True,
        style_function=lambda feature: 
               {'color': "black",
               'weight':1.5,
               'fillColor': colormap(gdf[feature['id']])}
              ).add_to(sfmap)
    folium.LayerControl().add_to(sfmap)
    
    #read in ambulance data, add markers
    
    ambulance = pd.read_csv(ambfile)
    
    #input ems event markers
    add_emsmarker(sfmap,current_call[1],current_call[2],current_call[0])
    
    #dispatch an ambulance
    available_amb = ambulance.loc[ambulance.AVAILABLE == 1]
    amb_coord = [(row[2], row[3]) for row in available_amb.itertuples()]
    call_coord = [(current_call[1], current_call[2])]
    dep_time = current_call[3]
    result = api_call(amb_coord,call_coord,dep_time,api_key,available_amb)
    
    print("DISPATCHED AMBULANCE #:",result)
    #update ambulance pd and save to file
    ambulance.AVAILABLE[ambulance.AMB_ID==result] = 0
    
    #don't want to edit anyones files just yet
    ambulance.to_csv(ambfile,index=False)
    
    #add ambulance markers
    for i in ambulance.values:
        if i[1] != 0:
            if i[3] == 1:
                add_ambmarker(sfmap,i[1], i[2],i[0])
        
            else:
                dispatch_ambmarker(sfmap,i[1], i[2],i[0])
    return sfmap

In [17]:
def activate_ambulance(amb,ambfile):
    ambulance = pd.read_csv(ambfile)
    ambulance.AVAILABLE[ambulance.AMB_ID==amb] = 1
    ambulance.to_csv(ambfile,index=False)

In [18]:
activate_ambulance('amb_10','ambulance_loc.csv')

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [19]:
current_call = get_current_call('calls.csv')
##must be changed for everyone's key
key = get_api_key('/Users/priyagupta/Desktop/key.txt')

map_test = create_map('sf_zcta/sf_zcta.shp','ambulance_loc.csv',current_call,key)

DISPATCHED AMBULANCE #: amb_10


/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
map_test